# Cohesion and coupling

Based on [Cohesion and Coupling](https://youtu.be/eiDyK_ofPPM) by ArjanCodes

## Cohesion
Function with strong cohesion has a clear responsibility and do one task.<br>
It makes the function easier to maintain, understand and reuse.

## Coupling
Measures of how dependant two parts of the code are on each other.<br>
High coupling means, that changing code in one part of the program, requires to update code in multiple places.<br>

### Example

In [1]:
import string
import random

class VehicleRegistry:
    
    def generate_vehicle_id(self, length):
        return ''.join(random.choices(string.ascii_uppercase, k=length))
    
    def generate_vehicle_licence(self, id):
        return (
            f"{id[:2]}"
            f"-{''.join(random.choices(string.digits, k=2))}"
            f"-{''.join(random.choices(string.ascii_uppercase, k=2))}"
        )

In [2]:
class Application:
    def register_vehicle(self, brand: string):
        # create a registry instance
        registry = VehicleRegistry()
        
        # generate a vehicle id of length 12
        vehicle_id = registry.generate_vehicle_id(12)
        
        # now generate a licence plate for the vehicle
        # using the first two characters of the vehicle_id
        licence_plate = registry.generate_vehicle_licence(vehicle_id)
        
        # compute the catalogue price
        catalogue_price = 0
        if brand == "Tesla Model 3":
            catalogue_price = 60000
        elif brand == "Volkswagen ID3":
            catalogue_price = 35000
        elif brand == "BMW 5":
            catalogue_price = 45000
        
        # compute the tax percentage 
        # (default 5% of the catalogue price, except for electric cars where it is 2%)
        tax_percentage = 0.05
        if brand == "Tesla Model 3" or brand == "Volkswagen ID3":
            tax_percentage = 0.02
        
        # compute payable tax
        payable_tax = tax_percentage * catalogue_price
            
        # print out the vehicle registration information
        print("Registration complete. Vehicle information:")
        print(f"Brand: {brand}")
        print(f"Id: {vehicle_id}")
        print(f"Licence plate: {licence_plate}")
        print(f"Payable tax: {payable_tax}")

In [3]:
app = Application()
app.register_vehicle("BMW 5")

Registration complete. Vehicle information:
Brand: BMW 5
Id: NIZWIEXIRDNU
Licence plate: NI-19-CH
Payable tax: 2250.0


In [4]:
app.register_vehicle("Tesla Model 3")

Registration complete. Vehicle information:
Brand: Tesla Model 3
Id: XNGWYFBBDQYB
Licence plate: XN-51-SY
Payable tax: 1200.0


<hr>

In the above code the `.register_vehicle()` does a lot of different things: 
* generates id and licence plate, 
* computes catalogue price, 
* computes tax percentage and tax to pay, 
* prints registration information

This means this method has very low cohesion as it has too many responsibilities.

High coupling means that if we change anything in the `VehicleRegistry` class, it would require to change `.register_vehicle()`.

As a result of low cohesion it is very difficult to add another vehicle brand

### Improving code

Look where the information is stored and how is it accessed. <br>
When that is known, then the code could be grouped around that. <br>
Results in lower coupling as the code is closer to the information it uses. <br>
Also allows to have functions and methods that are high cohesion because they can do only one thing with that particular information. <br>

In [5]:
import string
import random

class VehicleInfo:
        
    def __init__(self, brand: str, electric: bool, catalogue_price: int):
        self.brand = brand
        self.electric = electric
        self.catalogue_price = catalogue_price
    
    def compute_tax(self):
        tax_percentage = 0.05
        if self.electric:
            tax_percentage = 0.02
        return tax_percentage * self.catalogue_price
    
    def print(self):
        print(f"Brand: {self.brand}")
        print(f"Payable tax: {self.compute_tax()}")

        
class Vehicle:
        
    def __init__(self, id: str, licence_plate: str, info: VehicleInfo):
        self.id = id
        self.licence_plate = licence_plate
        self.info = info
    
    def print(self):
        print(f"ID: {self.id}")
        print(f"Licence plate: {self.licence_plate}")
        self.info.print()

In [6]:
class VehicleRegistry:
    
    vehicle_info = {}
    
    def add_vehicle_info(self, brand, electric, catalogue_price):
        self.vehicle_info[brand] = VehicleInfo(brand, electric, catalogue_price)
    
    def __init__(self):
        self.add_vehicle_info("Tesla Model 3", True, 60000)
        self.add_vehicle_info("Volkswagen ID3", True, 35000)
        self.add_vehicle_info("BMW 5", False, 45000)
    
    def generate_vehicle_id(self, length):
        return ''.join(random.choices(string.ascii_uppercase, k=length))
    
    def generate_vehicle_licence(self, id):
        return (
            f"{id[:2]}"
            f"-{''.join(random.choices(string.digits, k=2))}"
            f"-{''.join(random.choices(string.ascii_uppercase, k=2))}"
        )
    
    def create_vehicle(self, brand):
        vehicle_id = self.generate_vehicle_id(12)
        licence_plate = self.generate_vehicle_licence(vehicle_id)
        return Vehicle(vehicle_id, licence_plate, self.vehicle_info[brand])

In [7]:
class Application:
    def register_vehicle(self, brand: string):
        # create a registry instance
        registry = VehicleRegistry()
        
        # create a vehicle
        return registry.create_vehicle(brand)

In [8]:
app = Application()
vehicle = app.register_vehicle("BMW 5")
vehicle.print()

ID: EWBUQXUXFAQR
Licence plate: EW-46-UN
Brand: BMW 5
Payable tax: 2250.0


<hr>
Final UML:

<img src="images/cohesion_and_coupling_final.png" width=800>